In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.applications import ResNet50, DenseNet121, MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras.datasets import mnist

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess data
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)
x_train = tf.image.resize(x_train, (32, 32))
x_test = tf.image.resize(x_test, (32, 32))
x_train = tf.image.grayscale_to_rgb(x_train)
x_test = tf.image.grayscale_to_rgb(x_test)
x_train = x_train / 255.0
x_test = x_test / 255.0

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)
datagen.fit(x_train)

# Define input shape
input_shape = (32, 32, 3)

# ResNet Model
resnet_base = ResNet50(weights="imagenet", include_top=False, input_shape=input_shape, pooling="avg")
resnet_model = Sequential([
    resnet_base,
    Dense(128, activation="relu"),
    Dense(10, activation="softmax")
])
resnet_model.compile(optimizer=Adam(learning_rate=0.0001), loss=SparseCategoricalCrossentropy(), metrics=["accuracy"])

# DenseNet Model
densenet_base = DenseNet121(weights="imagenet", include_top=False, input_shape=input_shape, pooling="avg")
densenet_model = Sequential([
    densenet_base,
    Dense(128, activation="relu"),
    Dense(10, activation="softmax")
])
densenet_model.compile(optimizer=Adam(learning_rate=0.0001), loss=SparseCategoricalCrossentropy(), metrics=["accuracy"])

# MobileNet Model
mobilenet_base = MobileNetV2(weights="imagenet", include_top=False, input_shape=input_shape, pooling="avg")
mobilenet_model = Sequential([
    mobilenet_base,
    Dense(128, activation="relu"),
    Dense(10, activation="softmax")
])
mobilenet_model.compile(optimizer=Adam(learning_rate=0.0001), loss=SparseCategoricalCrossentropy(), metrics=["accuracy"])

# Callbacks
early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-6)

# Train ResNet
history_resnet = resnet_model.fit(
    datagen.flow(x_train, y_train, batch_size=32),
    validation_data=(x_test, y_test),
    epochs=20,
    callbacks=[early_stopping, lr_scheduler]
)

# Evaluate ResNet
resnet_test_loss, resnet_test_accuracy = resnet_model.evaluate(x_test, y_test, verbose=2)
print(f"ResNet Test Accuracy: {resnet_test_accuracy * 100:.2f}%")

# Train DenseNet
history_densenet = densenet_model.fit(
    datagen.flow(x_train, y_train, batch_size=32),
    validation_data=(x_test, y_test),
    epochs=20,
    callbacks=[early_stopping, lr_scheduler]
)

# Evaluate DenseNet
densenet_test_loss, densenet_test_accuracy = densenet_model.evaluate(x_test, y_test, verbose=2)
print(f"DenseNet Test Accuracy: {densenet_test_accuracy * 100:.2f}%")

# Train MobileNet
history_mobilenet = mobilenet_model.fit(
    datagen.flow(x_train, y_train, batch_size=32),
    validation_data=(x_test, y_test),
    epochs=20,
    callbacks=[early_stopping, lr_scheduler]
)

# Evaluate MobileNet
mobilenet_test_loss, mobilenet_test_accuracy = mobilenet_model.evaluate(x_test, y_test, verbose=2)
print(f"MobileNet Test Accuracy: {mobilenet_test_accuracy * 100:.2f}%")
